In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()


from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import pickle

from sklearn.metrics import roc_curve, roc_auc_score

from sklearn.metrics import precision_score, recall_score, accuracy_score

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import confusion_matrix

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

np.set_printoptions(suppress=True) # Suppress scientific notation where possible

from imblearn.over_sampling import SMOTE


from mlxtend.plotting import plot_decision_regions
%matplotlib inline

# make prettier plots
%config InlineBackend.figure_format = 'svg' 

In [2]:
with open("MVP_patientlevel_intubation.pkl", 'rb') as picklefile: 
    patients = pickle.load(picklefile)

patients.head()

,id,Age,Patient_Gender,Pneumonia,Pregnant,Indigenous,Diabetic,COPD_Lung_Disease,Asthma,Immunosuppressed,Hypertension,Comorbidity,Cardiovascular_Disease,Obese,Chronic_Renal_Insufficiency,Intensive_Care,Intubation
0,35483,50,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,7062,25,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
2,13779,29,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,17334,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2789,64,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0


In [3]:
patients.shape

(98820, 17)

In [7]:
# only 83 patients with COPD Lung Disease symptoms were intubated

len(patients.loc[(patients.COPD_Lung_Disease == 1) &(patients.Intubation == 1)])

83

In [8]:
df = pd.read_csv("https://raw.githubusercontent.com/marianarf/covid19_mexico_analysis/master/mexico_covid19.csv")
df.head()

,id,FECHA_ARCHIVO,ID_REGISTRO,ENTIDAD_UM,ENTIDAD_RES,RESULTADO,DELAY,ENTIDAD_REGISTRO,ENTIDAD,ABR_ENT,...,OTRA_COM,CARDIOVASCULAR,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
0,9269,2020-04-12,00011f,25,25,2,0,25,Sinaloa,SL,...,2,2,1,2,2,2,99,MÃ©xico,97,97
1,33333,2020-04-12,00014e,14,14,2,0,14,Jalisco,JC,...,2,2,1,2,1,99,99,MÃ©xico,97,2
2,35483,2020-04-12,000153,8,8,1,0,8,Chihuahua,CH,...,2,2,2,2,2,99,99,MÃ©xico,97,2
3,7062,2020-04-12,0001b6,9,15,1,0,9,Ciudad de Mexico,DF,...,2,2,1,2,2,99,99,MÃ©xico,97,97
4,23745,2020-04-12,0001c1,9,9,2,0,9,Ciudad de Mexico,DF,...,2,2,2,2,2,99,99,MÃ©xico,97,97


In [10]:
df.FECHA_ARCHIVO.min(), df.FECHA_ARCHIVO.max()

('2020-04-12', '2020-06-03')